In [4]:
from fastapi import FastAPI, HTTPException, Depends, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Optional
import pyodbc
import pandas as pd
from datetime import datetime, timedelta

app = FastAPI(title="DHL Logistics API")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # For development only - restrict in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Database connection
def get_connection():
    server = 'mcruebs04.isad.isadroot.ex.ac.uk'
    database = 'BEMM459_GroupO'
    username = 'GroupO'
    password = 'MrjV827*Wr'
    
    conn = pyodbc.connect(
        f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};',
        autocommit=True
    )
    try:
        yield conn
    finally:
        conn.close()

# Models
class Customer(BaseModel):
    CustomerID: int
    Name: str
    Email: str
    Phone: str
    Address: str
    Type: str

class Parcel(BaseModel):
    ParcelID: int
    ParcelName: str
    CustomerID: int
    Weight: float
    Type: str
    ShippingMethod: str

class ShipmentUpdate(BaseModel):
    Status: str
    CurrentLocation: str

class ShipmentAnalyticsResponse(BaseModel):
    ShipmentID: int
    Efficiency: float
    Delays: int
    CustomerRating: int
    
class DashboardSummary(BaseModel):
    total_shipments: int
    active_shipments: int
    delayed_shipments: int
    completed_shipments: int
    average_efficiency: float
    average_customer_rating: float

# API Routes
@app.get("/", tags=["Root"])
async def root():
    return {"message": "Welcome to DHL Logistics API"}

# Dashboard Summary
@app.get("/dashboard/summary", response_model=DashboardSummary, tags=["Dashboard"])
async def get_dashboard_summary(conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    
    # Get shipment counts
    cursor.execute("SELECT COUNT(*) FROM Shipments")
    total_shipments = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM Shipments WHERE Status = 'In Transit'")
    active_shipments = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM Shipments WHERE Status = 'Delayed'")
    delayed_shipments = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM Shipments WHERE Status = 'Delivered'")
    completed_shipments = cursor.fetchone()[0]
    
    # Get average efficiency and customer rating
    cursor.execute("SELECT AVG(Efficiency), AVG(CustomerRating) FROM Analytics")
    avg_data = cursor.fetchone()
    average_efficiency = avg_data[0]
    average_customer_rating = avg_data[1]
    
    cursor.close()
    
    return {
        "total_shipments": total_shipments,
        "active_shipments": active_shipments,
        "delayed_shipments": delayed_shipments,
        "completed_shipments": completed_shipments,
        "average_efficiency": average_efficiency,
        "average_customer_rating": average_customer_rating
    }

# Customer routes
@app.get("/customers", response_model=List[Customer], tags=["Customers"])
async def get_customers(limit: int = 50, conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute(f"SELECT TOP {limit} * FROM Customers")
    
    customers = []
    for row in cursor.fetchall():
        customers.append({
            "CustomerID": row.CustomerID,
            "Name": row.Name,
            "Email": row.Email,
            "Phone": row.Phone,
            "Address": row.Address,
            "Type": row.Type
        })
    
    cursor.close()
    return customers

@app.get("/customers/{customer_id}", response_model=Customer, tags=["Customers"])
async def get_customer(customer_id: int, conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Customers WHERE CustomerID = ?", (customer_id,))
    
    customer = cursor.fetchone()
    if not customer:
        raise HTTPException(status_code=404, detail="Customer not found")
    
    cursor.close()
    return {
        "CustomerID": customer.CustomerID,
        "Name": customer.Name,
        "Email": customer.Email,
        "Phone": customer.Phone,
        "Address": customer.Address,
        "Type": customer.Type
    }

# Parcel routes
@app.get("/parcels", response_model=List[Parcel], tags=["Parcels"])
async def get_parcels(limit: int = 50, conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute(f"SELECT TOP {limit} * FROM Parcels")
    
    parcels = []
    for row in cursor.fetchall():
        parcels.append({
            "ParcelID": row.ParcelID,
            "ParcelName": row.ParcelName,
            "CustomerID": row.CustomerID,
            "Weight": row.Weight,
            "Type": row.Type,
            "ShippingMethod": row.ShippingMethod
        })
    
    cursor.close()
    return parcels

@app.get("/parcels/{parcel_id}", response_model=Parcel, tags=["Parcels"])
async def get_parcel(parcel_id: int, conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Parcels WHERE ParcelID = ?", (parcel_id,))
    
    parcel = cursor.fetchone()
    if not parcel:
        raise HTTPException(status_code=404, detail="Parcel not found")
    
    cursor.close()
    return {
        "ParcelID": parcel.ParcelID,
        "ParcelName": parcel.ParcelName,
        "CustomerID": parcel.CustomerID,
        "Weight": parcel.Weight,
        "Type": parcel.Type,
        "ShippingMethod": parcel.ShippingMethod
    }

# Shipment routes
@app.get("/shipments", tags=["Shipments"])
async def get_shipments(
    status: Optional[str] = None, 
    limit: int = 50,
    conn: pyodbc.Connection = Depends(get_connection)
):
    cursor = conn.cursor()
    
    query = f"""
    SELECT TOP {limit} s.*, p.ParcelName, p.Type as ParcelType, c.Name as CustomerName
    FROM Shipments s
    JOIN Parcels p ON s.ParcelID = p.ParcelID
    JOIN Customers c ON p.CustomerID = c.CustomerID
    """
    
    if status:
        query += f" WHERE s.Status = '{status}'"
    
    cursor.execute(query)
    
    shipments = []
    for row in cursor.fetchall():
        shipments.append({
            "ShipmentID": row.ShipmentID,
            "ShipmentName": row.ShipmentName,
            "ParcelID": row.ParcelID,
            "ParcelName": row.ParcelName,
            "ParcelType": row.ParcelType,
            "CustomerName": row.CustomerName,
            "ShipmentDate": row.ShipmentDate.isoformat() if row.ShipmentDate else None,
            "DeliveryDate": row.DeliveryDate.isoformat() if row.DeliveryDate else None,
            "Status": row.Status,
            "CurrentLocation": row.CurrentLocation
        })
    
    cursor.close()
    return shipments

@app.put("/shipments/{shipment_id}", tags=["Shipments"])
async def update_shipment_status(
    shipment_id: int, 
    update: ShipmentUpdate,
    conn: pyodbc.Connection = Depends(get_connection)
):
    cursor = conn.cursor()
    
    # Check if shipment exists
    cursor.execute("SELECT * FROM Shipments WHERE ShipmentID = ?", (shipment_id,))
    if not cursor.fetchone():
        cursor.close()
        raise HTTPException(status_code=404, detail="Shipment not found")
    
    # Update shipment status
    cursor.execute(
        "UPDATE Shipments SET Status = ?, CurrentLocation = ? WHERE ShipmentID = ?",
        (update.Status, update.CurrentLocation, shipment_id)
    )
    
    cursor.close()
    return {"message": "Shipment updated successfully"}

# Analytics routes
@app.get("/analytics/shipments", response_model=List[ShipmentAnalyticsResponse], tags=["Analytics"])
async def get_shipment_analytics(conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Analytics")
    
    analytics = []
    for row in cursor.fetchall():
        analytics.append({
            "ShipmentID": row.ShipmentID,
            "Efficiency": row.Efficiency,
            "Delays": row.Delays,
            "CustomerRating": row.CustomerRating
        })
    
    cursor.close()
    return analytics

@app.get("/analytics/shipment-status-count", tags=["Analytics"])
async def get_shipment_status_count(conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("SELECT Status, COUNT(*) as Count FROM Shipments GROUP BY Status")
    
    result = []
    for row in cursor.fetchall():
        result.append({
            "status": row.Status,
            "count": row.Count
        })
    
    cursor.close()
    return result

@app.get("/analytics/efficiency-by-parcel-type", tags=["Analytics"])
async def get_efficiency_by_parcel_type(conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT p.Type, AVG(a.Efficiency) as AvgEfficiency, AVG(a.CustomerRating) as AvgRating
        FROM Parcels p
        JOIN Shipments s ON p.ParcelID = s.ParcelID
        JOIN Analytics a ON s.ShipmentID = a.ShipmentID
        GROUP BY p.Type
    """)
    
    result = []
    for row in cursor.fetchall():
        result.append({
            "parcelType": row.Type,
            "averageEfficiency": row.AvgEfficiency,
            "averageRating": row.AvgRating
        })
    
    cursor.close()
    return result

# Route visualization endpoints
@app.get("/route/locations", tags=["Route Visualization"])
async def get_route_locations(conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT CurrentLocation FROM Shipments")
    
    locations = []
    for row in cursor.fetchall():
        locations.append(row.CurrentLocation)
    
    cursor.close()
    return locations

@app.get("/route/active-shipments", tags=["Route Visualization"])
async def get_active_shipment_routes(conn: pyodbc.Connection = Depends(get_connection)):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT s.ShipmentID, s.ShipmentName, s.CurrentLocation, p.Type, p.Weight
        FROM Shipments s
        JOIN Parcels p ON s.ParcelID = p.ParcelID
        WHERE s.Status = 'In Transit'
    """)
    
    routes = []
    for row in cursor.fetchall():
        # For a real app, you would replace this with actual route data
        # This is just placeholder data for visualization
        routes.append({
            "shipmentId": row.ShipmentID,
            "shipmentName": row.ShipmentName,
            "currentLocation": row.CurrentLocation,
            "parcelType": row.Type,
            "weight": row.Weight,
            "routePoints": [
                {"x": 0, "y": 0},
                {"x": 50, "y": 100},
                {"x": 100, "y": 150},
                {"x": 150, "y": 100}
            ]
        })
    
    cursor.close()
    return routes

if __name__ == "__main__":
    import uvicorn
    import nest_asyncio
    from IPython.display import display, Markdown

    # Allow nested event loops in Jupyter Notebook
    nest_asyncio.apply()

    # Display server information
    display(Markdown("**Server is running at [http://127.0.0.1:8000](http://127.0.0.1:8000)**"))

    # Run the FastAPI app
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info", reload=False)

**Server is running at [http://127.0.0.1:8000](http://127.0.0.1:8000)**

INFO:     Started server process [39284]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [39284]
